<a href="https://colab.research.google.com/github/Naveensadanandan/Pytorch/blob/main/03_Computervision_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 2.3.1+cu121
torchvision version: 0.18.1+cu121


# Creating Data

In [ ]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download= True,
    transform=ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download= True,
    transform=ToTensor(),
    target_transform=None
)

 58%|█████▊    | 15204352/26421880 [00:06<00:02, 4537829.50it/s]

In [ ]:
len(train_data.targets),len(train_data.data),len(test_data.targets),len(test_data.data)

In [ ]:
train_data.classes,train_data.class_to_idx

In [ ]:
image, label = train_data[10]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze())
plt.title(label)
plt.show()

In [ ]:
# See classes
class_names = train_data.classes
class_names

In [ ]:
# Plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title(class_names[label])
    plt.axis(False);

# Preparing Dataloaders

In [ ]:
from torch.utils.data import DataLoader

# Setup the batch size hyperparameter
BATCH_SIZE = 32

train_data_loader = DataLoader(dataset = train_data,
                              batch_size = BATCH_SIZE,
                              shuffle=True)
test_data_loader = DataLoader(dataset = test_data,
                              batch_size = BATCH_SIZE,
                              shuffle=False)

train_data_loader, test_data_loader

In [ ]:
train_features_batch, train_label_batch = next(iter(train_data_loader))
train_features_batch.shape, train_label_batch.shape

In [ ]:
# Show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_label_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis("Off");
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

# Creating a Base line model

In [ ]:
torch_flatten = nn.Flatten()
x = train_features_batch[0]
output = torch_flatten(x)

print(f"shape before apllying flatten : {x.shape}")
print(f"shape after apllying flatten : {output.shape}")

In [ ]:
class FashionMnistModelV0(nn.Module):
  def __init__(self,
               input : int,
               hidden : int,
               output : int) -> None:
     super().__init__()
     self.linear_layer_stack = nn.Sequential(
         nn.Flatten(),
         nn.Linear(in_features=input, out_features=hidden),
         nn.Linear(in_features=hidden, out_features=output)
     )

  def forward(self,x):
    return self.linear_layer_stack(x)


modelV0 = FashionMnistModelV0(input = 28*28,
                              hidden = 8,
                              output = len(class_names)).to("cpu")

modelV0

In [ ]:
dummy = torch.rand(size = (1,28,28))
modelV0(dummy).shape

## downloading helper function

In [ ]:
import requests
from pathlib import Path

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optmizer = torch.optim.SGD(params = modelV0.parameters(), lr = 0.1)

In [ ]:
from timeit import default_timer as timer
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format).
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)
train_time_start_on_cpu = timer()
epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"epoch : {epoch}\n..............")
  train_loss = 0
  for batch , (X,y) in enumerate(train_data_loader):
    modelV0.train()

    y_pred = modelV0(X)

    loss = loss_fn(y_pred, y)

    train_loss += loss

    optmizer.zero_grad()

    loss.backward()

    optmizer.step()

    if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_data_loader.dataset)} samples")
  train_loss /= len(train_data_loader)

  modelV0.eval()

  with torch.inference_mode():
    test_loss, test_acc = 0, 0
    for (X_test,y_test) in test_data_loader:
      test_pred = modelV0(X_test)

      test_loss += loss_fn(test_pred, y_test)

      test_acc += accuracy_fn(y_true=y_test, y_pred = test_pred.argmax(dim=1))

    test_loss /= len(test_data_loader)

        # Divide total accuracy by length of test dataloader (per batch)
    test_acc /= len(test_data_loader)
  ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")

# Calculate training time
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                           end=train_time_end_on_cpu,
                                           device=str(next(modelV0.parameters()).device))



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device